In [ ]:
#Install and freeze required dependencies for ease of access
%pip install pandas
%pip install torch
%pip install datasets
%pip install transformers
%pip install openai
%pip install subprocess
%pip install sys
%pip freeze > requirements.txt

In [20]:
import pandas as pd
from datasets import load_dataset
from openai import OpenAI
import re

# Initialize
client = OpenAI(api_key = "api-key")

# Load
hi_tom_dataset = load_dataset("Hi-ToM/Hi-ToM_Dataset")
df = hi_tom_dataset["train"].to_pandas()
subset = df.head(100)

# Placeholder
responses = []
correctness = []

# Parse
def parse_ai_answer(response_content):
    match = re.search(r"(?:the correct answer is:\s*[A-Z]\.\s*)(.+?)(?:\.|$)", response_content, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return response_content.strip()

# Iterate through the subset
for index, row in subset.iterrows():
    question = row["question"]
    story = row["story"]
    correct_answer = row["answer"]
    choices = row.get("choices", "")
    prompt = (
        f"Read the following story and answer the multiple-choice question. Think step-by-step. "
        f"Provide the answer first, and then explain it \n\n"
        f"Story: {story}\n\n"
        f"Question: {question}\n\n"
        f"Choices: {choices}\n\n"
        f"Note: You should assume the following. (1) An agent witnesses everything and every movement before exiting a room. "
        f"(2) An agent A can infer another agent B’s mental state only if A and B have been in the same room, or have private or public interactions. "
        f"(3) Note that every agent tends to lie. What a character tells others doesn’t affect their actual belief. "
        f"An agent tends to trust an agent that exited the room later than themselves. The exit order is known to all agents. "
        f"(4) Agents in private communications know that others won’t hear them, but they know that anyone can hear any public claims."
    )
    
    # Call
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    # Extract
    raw_ai_answer = response.choices[0].message.content
    ai_answer = parse_ai_answer(raw_ai_answer)
    responses.append(ai_answer)
    
    # "Correctness"
    is_correct = ai_answer.strip().lower() == correct_answer.strip().lower()
    correctness.append(is_correct)

# Add the responses and correctness to the DataFrame
subset["ai_answer"] = responses
subset["is_correct"] = correctness

# Calculate and print the total percentage of correct answers
accuracy = sum(correctness) / len(correctness) * 100
print(f"Total Accuracy: {accuracy:.2f}%")

# Save the resulting DataFrame to a CSV file
subset.to_csv("hi_tom_responses_with_accuracy.csv", index=False)
# Confirm
print("Responses with accuracy saved to hi_tom_responses_with_accuracy.csv")



Repo card metadata block was not found. Setting CardData to empty.


Total Accuracy: 0.00%
Responses with accuracy saved to hi_tom_responses_with_accuracy.csv


C:\Users\mtptr\AppData\Local\Temp\ipykernel_23876\2804423484.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["ai_answer"] = responses
C:\Users\mtptr\AppData\Local\Temp\ipykernel_23876\2804423484.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["is_correct"] = correctness
